In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
import nipype.interfaces.mrtrix3 as mtx
import nipype.interfaces.fsl as fsl
from nipype.workflows.dmri.mrtrix.diffusion import create_mrtrix_dti_pipeline
from pandas import Series, read_csv, to_numeric
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd, makedirs
from shutil import copyfile
from nipype import config, logging
from datetime import date
import AFQ
today = str(date.today())
config.enable_debug_mode()

/usr/local/anaconda3/lib/python3.6/site-packages/nipype/workflows/dmri/mrtrix/group_connectivity.py:16: UserWarning: cmp not installed
  warnings.warn('cmp not installed')


In [2]:
#Set user and path variables

user = expanduser('~')
if user == '/Users/lucindasisk':
    home = join(user, 'Desktop/Milgram/candlab')
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    proc_dir = join(home, 'analyses/shapes/dwi/eddyCUDA_data')
    workflow_dir = join(home, 'analyses/shapes/dwi/tractography_workflow')
    data_dir = join(home, 'analyses/shapes/dwi/tractography_data')
else:
    home = '/gpfs/milgram/project/gee_dylan/candlab'
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    proc_dir = join(home, 'analyses/shapes/dwi/eddyCUDA_data')
    workflow_dir = join(home, 'analyses/shapes/dwi/tractography_workflow')
    data_dir = join(home, 'analyses/shapes/dwi/tractography_data')
    
# Read in subject subject_list
# subject_info = read_csv(
#     home + '/scripts/shapes/mri/dwi/shapes_dwi_subjList_08.07.2019.txt', sep=' ', header=None)
# subject_list = subject_info[0].tolist()

# Manual subject list
subject_list = ['sub-A200']  # , 'sub-A201']

In [3]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir,
                        substitutions = [('_subject_id_', '')]),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(dti = join(proc_dir,'3_Eddy_Corrected/{subject_id}/eddy_corrected.nii.gz'),
                bval = join(raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bval'),
                bvec = join(proc_dir,'3_Eddy_Corrected/{subject_id}/eddy_corrected.eddy_rotated_bvecs'),
                t1 = join(raw_dir, '{subject_id}/ses-shapesV1/anat/{subject_id}_ses-shapesV1_T1w.nii.gz')
               )

sf = Node(SelectFiles(template, 
                      base_directory = home),
          name = 'sf')
                

In [8]:
#Generate binary mask
bet=Node(fsl.BET(frac=0.6,
                mask=True),
        name='bet')

#Generate 5 tissue type (5tt) segmentation using FAST algorithm
seg5tt = Node(mtx.Generate5tt(algorithm = 'fsl',
                             out_file = 'T1s_5tt_segmented.nii.gz'),
             name='seg5tt')

#Estimate response functions for spherical deconvolution using the specified algorithm (Dhollander)
#https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/preprocess.html#responsesd
#https://mrtrix.readthedocs.io/en/latest/constrained_spherical_deconvolution/response_function_estimation.html#response-function-estimation
#Max_sh (lmax variable) determined in shell order from here: https://mrtrix.readthedocs.io/en/3.0_rc2/constrained_spherical_deconvolution/lmax.html
#DWI has 5 shells: 7 b0 volumes, 6 b500 vols, 15 b1000 vols, 15 b2000 bols, 60 b3000 vols
dwiresp = Node(mtx.ResponseSD(algorithm = 'dhollander',
                              max_sh=[2,2,4,4,8],
                              wm_file = 'wm_response.txt',
                              gm_file = 'gm_response.txt',
                              csf_file = 'csf_response.txt'),
              name='dwiresp')

#Estimate fiber orientation distributions from diffusion data sing spherical deconvolution
#https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/reconst.html
#https://mrtrix.readthedocs.io/en/latest/constrained_spherical_deconvolution/multi_shell_multi_tissue_csd.html
#Max SH here determined by tissue type - chose 8,8,8 per forum recommendations
mscsd = Node(mtx.EstimateFOD(algorithm = 'msmt_csd',
                             bval_scale = 'yes',
                            max_sh = [8,8,8]),
            name='mscsd')

#Perform Tractography - ACT using iFOD2 (https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/tracking.html) 
tract = Node(mtx.Tractography(algorithm='iFOD2',
                             n_trials=10000),
            name='tract')

In [9]:
tract_flow = Workflow(name = 'tract_flow')
tract_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
                    (sf, bet, [('t1', 'in_file')]),
                    (sf, seg5tt, [('t1', 'in_file')]),
                    (seg5tt, datasink, [('out_file', '4_tract_Reconstruction')]),
                    (sf, dwiresp, [('dti', 'in_file'),
                                   ('bval','in_bval'),
                                   ('bvec', 'in_bvec')]),
                    (dwiresp, datasink, [('wm_file', '4_tract_Reconstruction.@par'),
                                        ('gm_file', '4_tract_Reconstruction.@par.@par'),
                                        ('csf_file', '4_tract_Reconstruction.@par.@par.@par')]),
                    (sf, mscsd, [('dti', 'in_file'),
                                ('bval', 'in_bval'),
                                ('bvec', 'in_bvec')]),
                    (dwiresp, mscsd, [('wm_file', 'wm_txt'),
                                      ('gm_file', 'gm_txt'),
                                      ('csf_file', 'csf_txt')]),
                    (seg5tt, tract, [('out_file', 'act_file')]),
                    (mscsd, tract, [('wm_odf', 'in_file')]),
                    (sf, tract, [('bval', 'in_bval'),
                                 ('bvec', 'in_bvec')]),
                    (bet, tract, [('mask_file', 'seed_image')]),
                    (tract, datasink, [('out_file', '4_tract_Reconstruction.@par.@par.@par.@par'),
                                     ('out_seeds','4_tract_Reconstruction.@par.@par.@par.@par.@par')])
                   ])
tract_flow.base_dir = workflow_dir
tract_flow.write_graph(graph2use = 'flat')
dwi = tract_flow.run('MultiProc', plugin_args={'n_procs': 4})

190813-15:14:20,688 nipype.workflow DEBUG:
	 (tract_flow.infosource, tract_flow.sf): No edge data
190813-15:14:20,689 nipype.workflow DEBUG:
	 (tract_flow.infosource, tract_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
190813-15:14:20,690 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.bet): No edge data
190813-15:14:20,691 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.bet): new edge data: {'connect': [('t1', 'in_file')]}
190813-15:14:20,691 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.seg5tt): No edge data
190813-15:14:20,692 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.seg5tt): new edge data: {'connect': [('t1', 'in_file')]}
190813-15:14:20,693 nipype.workflow DEBUG:
	 (tract_flow.seg5tt, tract_flow.datasink): No edge data
190813-15:14:20,694 nipype.workflow DEBUG:
	 (tract_flow.seg5tt, tract_flow.datasink): new edge data: {'connect': [('out_file', '4_tract_Reconstruction')]}
190813-15:14:20,694 nipype.workflow DEBUG:
	 (tract_flow.sf, t

190813-15:14:23,217 nipype.workflow DEBUG:
	 [Node] Up-to-date cache found for "tract_flow.sf".
190813-15:14:23,218 nipype.workflow DEBUG:
	 Checking hash "tract_flow.sf" locally: cached=True, updated=True.
190813-15:14:23,234 nipype.workflow DEBUG:
	 [MultiProc] Submitted task tract_flow.sf (taskid=1).
190813-15:14:23,240 nipype.workflow INFO:
	 [Node] Setting-up "tract_flow.sf" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/sf".
190813-15:14:23,264 nipype.workflow DEBUG:
	 [Node] Hashes: [('base_directory', '/Users/lucindasisk/Desktop/Milgram/candlab'), ('force_lists', False), ('raise_on_empty', True), ('sort_filelist', True), ('subject_id', 'sub-A200')], b55969a6e4f3187f1953032d2972e1b0, /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/sf/_0xb55969a6e4f3187f1953032d2972e1b0.json, ['/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi

190813-15:14:27,363 nipype.workflow DEBUG:
	 Skipping cached node tract_flow.dwiresp with ID 1.
190813-15:14:27,363 nipype.workflow INFO:
	 [Job 1] Cached (tract_flow.dwiresp).
190813-15:14:27,365 nipype.workflow DEBUG:
	 Allocating tract_flow.seg5tt ID=3 (0.20GB, 1 threads). Free: 28.40GB, 2 threads.
190813-15:14:27,366 nipype.workflow DEBUG:
	 Setting node inputs
190813-15:14:27,366 nipype.workflow DEBUG:
	 input: in_file
190813-15:14:27,367 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/sf/result_sf.pklz
190813-15:14:27,367 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/sf/result_sf.pklz
190813-15:14:27,398 nipype.workflow DEBUG:
	 output: t1
190813-15:14:27,399 nipype.workflow DEBUG:
	 [Node] seg5tt - setting input in_file = /Users/lucindasisk/Desktop/Milgram/candlab/data

190813-15:14:29,212 nipype.workflow DEBUG:
	 input: csf_txt
190813-15:14:29,213 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/result_dwiresp.pklz
190813-15:14:29,213 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/result_dwiresp.pklz
190813-15:14:29,235 nipype.workflow DEBUG:
	 output: csf_file
190813-15:14:29,235 nipype.workflow DEBUG:
	 [Node] mscsd - setting input csf_txt = /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/csf_response.txt
190813-15:14:29,502 nipype.workflow DEBUG:
	 [Node] Hashes: [('algorithm', 'msmt_csd'), ('bval_scale', 'yes'), ('csf_odf', 'csf.mif'), ('csf_txt', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_su

dwi2fod -bvalue_scaling yes -fslgrad /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.eddy_rotated_bvecs /Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/dwi/sub-A200_ses-shapesV1_dwi.bval -lmax 8,8,8 msmt_csd /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.nii.gz /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/wm_response.txt wm.mif /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/gm_response.txt gm.mif /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/csf_response.txt csf.mif
190813-15:14:31,506 nipype.interface DEBUG:
	 algorithm_msmt_csd
190813-15:14:31,511 nipype.interface DEBUG

...

tckgen -act /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/seg5tt/T1s_5tt_segmented.nii.gz -algorithm iFOD2 -fslgrad /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.eddy_rotated_bvecs /Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/dwi/sub-A200_ses-shapesV1_dwi.bval -samples 4 -trials 10000 -output_seeds out_seeds.nii.gz -seed_image /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/bet/sub-A200_ses-shapesV1_T1w_brain_mask.nii.gz /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/mscsd/wm.mif tracked.tck
190813-16:41:35,351 nipype.interface DEBUG:
	 act_file_/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/se

190813-16:43:09,68 nipype.workflow DEBUG:
	 output: out_file
190813-16:43:09,69 nipype.workflow DEBUG:
	 [Node] datasink - setting input 4_tract_Reconstruction.@par.@par.@par.@par = /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/tract/tracked.tck
190813-16:43:09,70 nipype.workflow DEBUG:
	 input: 4_tract_Reconstruction.@par.@par.@par.@par.@par
190813-16:43:09,70 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/tract/result_tract.pklz
190813-16:43:09,71 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/tract/result_tract.pklz
190813-16:43:09,91 nipype.workflow DEBUG:
	 output: out_seeds
190813-16:43:09,92 nipype.workflow DEBUG:
	 [Node] datasink - setting input 4_tract_Reconstruction.@par.@par.@par.@par.@par = <u

            * _outputs: [('4_tract_Reconstruction', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/seg5tt/T1s_5tt_segmented.nii.gz', '8982aa4321407f5d9b67cb1b444b3e72')), ('4_tract_Reconstruction.@par', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/wm_response.txt', 'e67e06cbc0d916f5516af022565d7886')), ('4_tract_Reconstruction.@par.@par', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/gm_response.txt', '8860fd89e6e0380083a57c149cbe6ff4')), ('4_tract_Reconstruction.@par.@par.@par', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/csf_response.txt', '1aeabd6cf5ec78ec75cb500e837cba4f')), ('4_tract_Reconstruction.@par.@par.@par.@par', ('/Users/lucindasisk/Desktop/Milgram/candlab/anal

	 File: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A200/wm_response.txt already exists,eccf025dafa4835ac78fe383d65a68b2, copy:1
190813-16:43:18,444 nipype.utils DEBUG:
	 File: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A200/wm_response.txt already exists, not overwriting, copy:1
190813-16:43:18,450 nipype.interface DEBUG:
	 key: 4_tract_Reconstruction.@par.@par files: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/gm_response.txt
190813-16:43:18,454 nipype.interface DEBUG:
	 sub.str: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/_subject_id_sub-A200/gm_response.txt -> /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A200/gm_response.txt using '_subject_id_' -> ''
190

	 Needed files: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A200/T1s_5tt_segmented.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A200/wm_response.txt;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A200/gm_response.txt;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A200/csf_response.txt;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A200/tracked.tck;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/seg5tt/T1s_5tt_segmented.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/wm_response.txt;/Users/lucindasisk/Desktop/Milgram/candlab/

In [ ]:
# dwi_flow = Workflow(name = 'dwi_flow')
# dwi_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
#                   (sf, mrconv, [('dti', 'in_file')]),
#                   (mrconv, dwi_res, [('out_file', 'in_file')]),
#                   (sf, dwi_res, [('bval', 'in_bval'),
#                                  ('bvec', 'in_bvec')]),
#                   (dwi_res, datasink, [('csf_file', '4_DWI_Reconstruction.@par'),
#                                       ('wm_file', '4_DWI_Reconstruction.@par.@par'),
#                                       ('gm_file', '4_DWI_Reconstruction.@par.@par.@par')]),
#                   (sf, ms_csd, [('dti','in_file'),
#                                ('bval','in_bval'),
#                                ('bvec','in_bvec'),
#                                ('mask', 'mask_file')]),
#                   (dwi_res, ms_csd, [('wm_file', 'wm_txt'),
#                                      ('gm_file', 'gm_txt'),
#                                      ('csf_file', 'csf_txt')]),
#                   (ms_csd, datasink, [('wm_odf', '4_DWI_Reconstruction.@par.@par.@par.@par'),
#                                      ('gm_odf','4_DWI_Reconstruction.@par.@par.@par.@par.@par'),
#                                      ('csf_odf', '4_DWI_Reconstruction.@par.@par.@par.@par.@par.@par')]) 
#                  ])
# dwi_flow.base_dir = workflow_dir
# dwi_flow.write_graph(graph2use = 'flat')
# dwi = dwi_flow.run('MultiProc', plugin_args={'n_procs': 4})